### Get access token
[Documentation](https://documentation.dataspace.copernicus.eu/APIs/Token.html)




### OAuth Setup
[Docs](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html#python), [Credentials](https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings)



In [32]:
%load_ext autoreload
%autoreload 2

import json
from copernicus import Client
credentials = json.load(open('credentials.json'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
client = Client(credentials['username'], credentials['password'])

In [34]:
aoi = "POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))"

client.search("2020-01-01", "2020-01-02", "SENTINEL-2", aoi)

https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-2' and ContentDate/Start gt 2020-01-01T00:00:00.000Z and ContentDate/Start lt 2020-01-02T00:00:00.000Z and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))')


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,37fe86b3-3953-5214-a8ad-e2e7300a265c,S2A_MSIL2A_20200101T105441_N0213_R051_T31UES_2...,application/octet-stream,0,2020-01-01T16:25:14.866Z,2020-01-01T21:12:20.728Z,2020-01-01T21:12:20.728Z,True,,/eodata/Sentinel-2/MSI/L2A/2020/01/01/S2A_MSIL...,[],"{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((2.9997121758513 ...,"{'type': 'Polygon', 'coordinates': [[[2.999712..."
1,application/octet-stream,2a598001-8248-48cd-a337-3bac87ec690c,S2A_MSIL1C_20200101T105441_N0500_R051_T31UES_2...,application/octet-stream,861556624,2023-10-21T08:49:58.825Z,2023-10-21T21:54:05.882Z,2023-10-21T21:54:15.623Z,True,,/eodata/Sentinel-2/MSI/L1C_N0500/2020/01/01/S2...,"[{'Value': 'd289cd005bdd7acb675c1ee7045bf334',...","{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((2.9997121758513 ...,"{'type': 'Polygon', 'coordinates': [[[2.999712..."
2,application/octet-stream,147580e0-de2b-4f89-a810-7b5c5b217fc8,S2A_MSIL2A_20200101T105441_N0500_R051_T31UES_2...,application/octet-stream,1110570408,2023-10-21T08:55:19.760Z,2023-10-21T22:00:05.591Z,2023-10-21T22:00:19.019Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2020/01/01/S2...,"[{'Value': '842486bdb292d8dadfb56e700d984963',...","{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((2.9997121758513 ...,"{'type': 'Polygon', 'coordinates': [[[2.999712..."
3,application/octet-stream,06734963-44cd-5b19-a36e-b265b7476b28,S2A_MSIL1C_20200101T105441_N0208_R051_T31UES_2...,application/octet-stream,0,2020-01-01T13:41:04.047Z,2020-01-01T21:07:13.822Z,2020-01-01T21:07:13.822Z,True,,/eodata/Sentinel-2/MSI/L1C/2020/01/01/S2A_MSIL...,[],"{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((2.9997121758513 ...,"{'type': 'Polygon', 'coordinates': [[[2.999712..."
4,application/octet-stream,80f885e7-9851-541a-ab2e-eaa299d9df6c,S2A_MSIL2A_20200101T105441_N0213_R051_T31UFS_2...,application/octet-stream,0,2020-01-01T16:31:08.667Z,2020-01-01T21:12:20.373Z,2020-01-01T21:12:20.373Z,True,,/eodata/Sentinel-2/MSI/L2A/2020/01/01/S2A_MSIL...,[],"{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((5.9651311350891 ...,"{'type': 'Polygon', 'coordinates': [[[5.965131..."
5,application/octet-stream,5e0689eb-2ef5-48ba-9fe4-5670d20a01f0,S2A_MSIL2A_20200101T105441_N0500_R051_T31UFS_2...,application/octet-stream,1156620310,2023-10-21T08:51:08.471Z,2023-10-21T21:57:20.835Z,2023-10-21T21:57:32.691Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2020/01/01/S2...,"[{'Value': 'd7bec019c811f7af5b7a5b768fe0ea47',...","{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((5.96512849911278...,"{'type': 'Polygon', 'coordinates': [[[5.965128..."
6,application/octet-stream,15f7d4a0-529c-4dff-b318-6e6d249db131,S2A_MSIL1C_20200101T105441_N0500_R051_T31UFS_2...,application/octet-stream,855065216,2023-10-21T08:49:03.385Z,2023-10-21T21:51:36.615Z,2023-10-21T21:51:45.781Z,True,,/eodata/Sentinel-2/MSI/L1C_N0500/2020/01/01/S2...,"[{'Value': '06fc20e0bc9787259f699749f21b84ec',...","{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((5.96512849911278...,"{'type': 'Polygon', 'coordinates': [[[5.965128..."
7,application/octet-stream,bff7ae33-7a82-5251-bb1e-0607188c0a4f,S2A_MSIL1C_20200101T105441_N0208_R051_T31UFS_2...,application/octet-stream,0,2020-01-01T13:44:02.988Z,2020-01-01T21:08:01.995Z,2020-01-01T21:08:01.995Z,True,,/eodata/Sentinel-2/MSI/L1C/2020/01/01/S2A_MSIL...,[],"{'Start': '2020-01-01T10:54:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((5.9651311350891 ...,"{'type': 'Polygon', 'coordinates': [[[5.965131..."
8,application/octet-stream,a73cee62-9333-569a-8516-7bcbb6c18884,S2A_MSIL2A_20200101T105441_N9999_R051_T31UFS_2...,application/octet-stream,1212591985,2022-09-27T01:

In [35]:
string ="filter=Collection/Name eq SENTINEL-2 and ContentDate/Start gt 2020-01-01T00:00:00.000Z and ContentDate/Start lt 2020-01-02T00:00:00.000Z and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))')"

In [36]:
string[27:]

"ENTINEL-2 and ContentDate/Start gt 2020-01-01T00:00:00.000Z and ContentDate/Start lt 2020-01-02T00:00:00.000Z and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))')"